In [ ]:
##================================================
## このセルを最初に実行せよ---Run this cell first.
##================================================
!wget -P . https://raw.githubusercontent.com/UTokyo-IPP/annex/master/assignments/project1/corpus_stoplist.txt
!wget -P . https://raw.githubusercontent.com/UTokyo-IPP/annex/master/assignments/project1/test.json
!wget -P . https://raw.githubusercontent.com/UTokyo-IPP/annex/master/assignments/project1/train.json

--2022-05-10 10:52:55--  https://raw.githubusercontent.com/UTokyo-IPP/annex/master/assignments/project1/corpus_stoplist.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 587 [text/plain]
Saving to: ‘./corpus_stoplist.txt’

corpus_stoplist.txt 100%[===================>]     587  --.-KB/s    in 0s      

2022-05-10 10:52:56 (9.08 MB/s) - ‘./corpus_stoplist.txt’ saved [587/587]

--2022-05-10 10:52:56--  https://raw.githubusercontent.com/UTokyo-IPP/annex/master/assignments/project1/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
L

# ミニプロジェクト（基礎課題）

[English version](https://colab.research.google.com/drive/1WXG_uhFclN5yHDnTcazZZR1LKK8KT_oe)

## Project1. 小説の著者をあててみよう（基礎課題）

[青空文庫](https://www.aozora.gr.jp/index.html)から、
以下に挙げる6名の著者の作品データをダウンロードし、データセット $D$ を作りました。

- 太宰治
- 宮沢賢治
- 夏目漱石
- 林芙美子
- 芥川龍之介
- 坂口安吾

このデータセット $D$ を使って、
著者が不明な小説（以下では、単に未知の小説と呼びます）はどの著者の作品かを推測しようと思います。
未知の小説は上記６名の著者のいずれかによって執筆された小説であるとします。  

このような文書分類の手法はいろいろありますが、今回は以下のような手順で行います。

1. $D$ の小説と未知の小説から重要な単語（**重要語**）を抜き出し、重要語リストを作成する。ここで、指定された単語（**ストップワード**）は重要語とはしない。
2. $D$ の小説に現れる重要語をキー、$D$ におけるその頻度（すなわち出現回数）を値とする辞書 $W$ を作成する。
3. $W$ では単語が多すぎるため、$W$ から頻度の降順に単語を選び、$D$ の重要語の総頻度の50%をカバーするような単語のリスト $F$ を作成する。
4. $D$ の各著者の小説、および未知の小説について、リスト $F$ の各単語が何回現れるかを数えることにより、**文書ベクトル**（Project1-3 参照）を作成する。
5. 未知の小説について、4で計算した文書ベクトルを使って $D$ の各著者と比べ、最も類似度の高い著者をその小説の著者と推測する。なお、文書ベクトルの比較にはコサイン類似度を用いる。

自然言語テキストをコンピュータで解析する際は、
テキストの語順や文法を考慮せず、語の集合として扱う
bag of words (BoW) という方法がよく使われます。
今回はこの方法を使って文書の類似度を評価します。

では、以上の手順を以下の課題を順に解くことで実装してください。

なお、以下の課題では、大文字で始まるいくつかの変数（`A`, `T`, `Aw`, `Dw` など）はグローバル変数として使われています。
これらの変数は、課題をまたがって使用されますので、注意してください。
一方、Project1-2 の `dw` など、関数の引数は小文字で始まります。
`Dw` と `dw` は別の変数ですので、注意してください。

### 注意事項

提出ファイルは1 MBを超えないようにしてください。大量の中間結果を`print`で書き出すようなコードは止めてください。

### ライセンス

本課題の作品データは[青空文庫](https://www.aozora.gr.jp/index.html)のものを使用しています。
記して感謝の意を表します。
なお、ルビや入力者注、アクセント分解された欧文や編者による注記等は削除しました。
また、著者間で学習データの文字数が概ね同じになるよう、作品数を調整しています。

クリエイティブコモンズ 表示 - 非営利 2.1 日本 (CC BY-NC 2.1 JP) の下にライセンスされています。
詳しくは[こちら](https://www.aozora.gr.jp/guide/kijyunn.html)を参照してください。

![CC-BY-NC_icon.svg](https://licensebuttons.net/l/by-nc/3.0/88x31.png)

## Project1-1. 品詞による重要語の抽出

### 準備0：著者が未知の小説集合の読み込み

JSONファイル `test.json` は判別対象である著者が未知の小説12作品を含みます。
以下のコードを用いて、6名の著者の小説を、各小説のタイトルをキー、その本文の（パラグラフの）文字列のリストを値とする辞書 `A` に読み込んでください。

In [ ]:
### Given code:
import json

A = {}
with open('test.json', 'r', encoding='utf-8') as f:
    A = json.load(f)

`A` には以下に示す小説が読み込まれています。

In [ ]:
for title in A:
    print(title)

老ハイデルベルヒ
八十八夜
毒もみのすきな署長さん
タネリはたしかにいちにち噛んでいたようだった
永日小品
變な音
蛙
恋愛の微醺
おぎん
闇中問答
ぐうたら戦記
桜枝町その他


試しに小説「老ハイデルベルヒ」の本文を書き出してみましょう。

In [ ]:
print(A['老ハイデルベルヒ'])

['八年まえの事でありました。当時、私は極めて懶惰な帝国大学生でありました。一夏を、東海道三島の宿で過したことがあります。五十円を故郷の姉から、これが最後だと言って、やっと送って戴き、私は学生鞄に着更の浴衣やらシャツやらを詰め込み、それを持ってふらと、下宿を立ち出で、そのまま汽車に乗りこめばよかったものを、方角を間違え、馴染みのおでんやにとびこみました。其処には友達が三人来合わせて居ました。やあ、やあ、めかして何処へ行くのだと、既に酔っぱらっている友人達は、私をからかいました。私は気弱く狼狽して、いや何処ということもないんだけど、君たちも、行かないかね、と心にも無い勧誘がふいと口から辷り出て、それからは騎虎の勢で、僕にね、五十円あるんだ、故郷の姉から貰ったのさ、これから、みんなで旅行に出ようよ、なに、仕度なんか要らない、そのままでいいじゃないか、行こう、行こう、とやけくそになり、しぶる友人達を引張るようにして連れ出してしまいました。あとは、どうなることか、私自身にさえわかりませんでした。あの頃は私も、随分、呑気なところのある子供でした。世の中も亦、私達を呑気に甘えさせてくれていました。私は、三島に行って小説を書こうと思って居たのでした。三島には高部佐吉さんという、私より二つ年下の青年が酒屋を開いて居たのです。佐吉さんの兄さんは沼津で大きい造酒屋を営み、佐吉さんは其の家の末っ子で、私とふとした事から知合いになり、私も同様に末弟であるし、また同様に早くから父に死なれている身の上なので、佐吉さんとは、何かと話が合うのでした。佐吉さんの兄さんとは私も逢ったことがあり、なかなか太っ腹の佳い方だし、佐吉さんは家中の愛を独占して居るくせに、それでも何かと不平が多い様で、家を飛出し、東京の私の下宿へ、にこにこ笑ってやって来た事もありました。さまざま駄々をこねて居たようですが、どうにか落ち附き、三島の町はずれに小ぢんまりした家を持ち、兄さんの家の酒樽を店に並べ、酒の小売を始めたのです。二十歳の妹さんと二人で住んで居ました。私は、其の家へ行くつもりであったのです。佐吉さんから、手紙で様子を聞いているだけで、まだ其の家を見た事も無かったので、行ってみて具合が悪いようだったらすぐ帰ろう、具合がいいようだったら一夏置いて貰って、小説を一篇書こう、そう思って居たのでありましたが、心ならずも三人

### 準備1：形態素解析

小説から文書ベクトルを作成する前準備として、小説を単語に分割し、
それぞれの単語の品詞や原形を特定します。
（小説を分割して得られるのは「形態素」ですが、簡単のためここでは「単語」と呼ぶことにします。）
そのためのツールとして、ここでは [janome](https://mocobeta.github.io/janome/) という形態素解析ツールを利用します。

形態素解析や自然言語処理について詳しく知りたい方は、S2期、A2期で開講予定の「メディアプログラミング入門」を受講してください。  
次のセルを実行して janome をColab上にインストールしてください。

In [ ]:
!pip install janome

     |████████████████████████████████| 19.7 MB 1.3 MB/s 


次に、以下のセルを実行し、形態素解析を行うためのエンジンを起動します。
エンジンは `Tokenizer` というクラスのオブジェクトです。
エンジンは何度も使いまわせるので、以下のセルは、このノートブックを開いた後で1度行えばよいです。
（ノートブックを再開したりカーネルを再起動したりした際は、エンジンも再度起動する必要があります。）

In [ ]:
### Execute only one time after this notebook starts up
from janome.tokenizer import Tokenizer

T = Tokenizer() # この処理は重いので一度得たTを使いまわす

試しに形態素解析を行ってみましょう。
日本語文の文字列 `s` を引数として、
先ほど起動したエンジン `T` の `tokenize(s)` というメソッドを呼び出すと、
解析結果としてトークンのリストが返されます。

トークン `token` は `Token` クラスのオブジェクトであり、形態素を表します。
`token.surface` でトークンの表層形（もとの文に現れる形）の文字列、
`token.part_of_speech` でトークンの品詞を表す文字列、
`token.base_form` でトークンの原形の文字列が得られます。 
これらは、`Token` クラスのオブジェクトの属性です。

したがって、以下のようにして形態素解析の結果を取り出すことができます。

In [ ]:
### Sample code of janome:

for token in T.tokenize('花が美しく咲きました。'):
    print('表層形:', token.surface)       # 表層形の書き出し
    print('品詞：', token.part_of_speech) # 品詞の書き出し
    print('原形:', token.base_form)       # 原形の書き出し
    print('---')

表層形: 花
品詞： 名詞,一般,*,*
原形: 花
---
表層形: が
品詞： 助詞,格助詞,一般,*
原形: が
---
表層形: 美しく
品詞： 形容詞,自立,*,*
原形: 美しい
---
表層形: 咲き
品詞： 動詞,自立,*,*
原形: 咲く
---
表層形: まし
品詞： 助動詞,*,*,*
原形: ます
---
表層形: た
品詞： 助動詞,*,*,*
原形: た
---
表層形: 。
品詞： 記号,句点,*,*
原形: 。
---


### 準備2：ストップワード

テキスト処理においては、一般語や高頻出語など、処理において必ずしも有用ではない単語（ストップワード）を予め除くことがあります。ストップワードの一覧をストップリストと呼びます。

以下では、本課題の小説の高頻度語から作成したストップリストのファイル `corpus_stoplist.txt` を使用します。`corpus_stoplist.txt` には各行にストップワードが1つずつ入っています。`corpus_stoplist.txt` を読み込んで、ストップワードを要素とするリスト `stop_list` を作成します。

In [ ]:
stop_list = []
with open('corpus_stoplist.txt', 'r', encoding='utf-8') as f:
        for line in f:
            stop_list.append(line.rstrip('\n'))

試しにストップワード100語を書き出してみましょう。

In [ ]:
print(stop_list[:100])

['する', 'の', '私', 'なる', '人', 'ある', 'よう', 'ない', 'ゐる', 'もの', '一', '事', 'こと', 'それ', '二', '云う', '見る', '何', 'つて', 'お', 'ん', '僕', '十', '中', '思う', 'いる', 'さん', '三', '時', 'やう', '彼', '日', '出る', '上', 'これ', '君', '行く', 'さ', '方', '女', 'ひる', '自分', '顔', '云', '来る', '家', 'ふる', '前', 'ところ', '男', '言う', 'たち', '五', 'そう', '気', 'いう', '知る', '先生', 'あなた', '眼', '年', '見える', 'いい', '声', '手', '御', 'みんな', 'つく', '持つ', '聞く', '四', '今', '者', 'やる', '風', '子供', '誰', '目', '話', 'ぢ', '出来る', '人間', '立つ', '主人', '大', 'あつい', '的', 'どこ', '歩く', '知れる', '思ふ', 'うち', '書く', '心', 'お前', '頭', 'ため', '出す', 'そこ', '六']


### 課題

小説の中の文字列 `doc` とストップワードを要素とするリスト `stop` をもらって、
重要語のリストを返す関数 `str2words(doc, stop)` を定義してください。

なお、重要語は以下のようにして得られるとします。

- 形態素解析を用いて、与えられた小説の文字列から、重要な単語（重要語）を抜き出します。ここでは、形容詞と動詞と名詞を重要と考えます。トークンの `part_of_speech` 属性が、文字列 `'形容詞,自立'`, `'動詞,自立'`, `'名詞'` のいずれかを（部分文字列として）含めば、トークンの**原形**を重要語と定義します。なお、品詞の中には`'接頭詞,名詞接続'`（例：「アンチ〇〇」の「アンチ」）があり、これは`'名詞'`ではないですが、ここではこれも含めて重要語として取り出してください。
- 上記の品詞を満たす語でもストップリストに含まれる語は重要語から除きます。

重要語（ただしストップリストに含まれる語は除かれたもの）は小説に現れる順でリストに現れるようにしてください。
また、関数の定義の中で形態素解析のエンジンを値とするグローバル変数 `T` を参照してください。

In [ ]:
##########################################################
##  <[ project1-1-str2words ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

QUESTION_EXISTS = False # 質問がある場合は True に変えて，このセル内のコメントとして質問を記述してください
                        # Change to True if you have questions, and describe them as comments in this cell

def str2words(doc, stop):
  word_list = []
  for token in T.tokenize(doc):
    for sub in ['形容詞,自立', '動詞,自立', '名詞']:
      if sub in token.part_of_speech and token.base_form not in stop:
        word_list.append(token.base_form)
  return word_list

提出前に以下のテストセルを実行し、エラーがでないことを確認してください。

In [ ]:
assert str2words('当時、私は極めて懶惰な帝国大学生でありました。', ['する','の','私','なる','人','ある']) == ['当時', '懶惰', '帝国', '大学生']
assert str2words(A['老ハイデルベルヒ'][0][59:176], stop_list) == \
      ['円', '故郷', '姉', '最後', '送る', '戴く', '学生', '鞄', '着る', '更', '浴衣', 'シャツ', '詰め込む',
       '下宿', '出', '汽車', '乗りこむ', '方角', '間違える', '馴染み', 'おでん', 'とびこむ']

## Project1-2. 頻出単語リストの作成

### 準備0：重要語リストへの変換と青空文庫データセットの読み込み

project1-1の `str2words` を用いて小説の本文に対応する重要語リストを値とする（Pythonの意味での）辞書 `Aw` を作成します。

In [ ]:
### Given code:

Aw = {}
for title in A:
    Aw[title] = []
    for string in A[title]:
        Aw[title].extend(str2words(string, stop_list))

試しに小説 `老ハイデルベルヒ` の重要語100語を書き出してみましょう。

In [ ]:
print(Aw['老ハイデルベルヒ'][:100])

['八', 'まえる', '当時', '懶惰', '帝国', '大学生', '夏', '東海道', '三島', '宿', '過す', '円', '故郷', '姉', '最後', '送る', '戴く', '学生', '鞄', '着る', '更', '浴衣', 'シャツ', '詰め込む', '下宿', '出', '汽車', '乗りこむ', '方角', '間違える', '馴染み', 'おでん', 'とびこむ', '其処', '友達', '来', '合わせる', '居る', 'めかす', '何処', '酔っぱらう', '友人', '達', 'からかう', '弱い', '狼狽', '何処', '君たち', '無い', '勧誘', 'ふく', '口', '辷る', '騎虎', '勢', '円', '故郷', '姉', '貰う', '旅行', 'なに', '仕度', '要る', 'やけくそ', 'しぶる', '友人', '達', '引張る', '連れ出す', 'あと', '自身', 'わかる', '頃', '呑気', '世の中', '達', '呑気', '甘える', '三島', '小説', '居る', '三島', '高部', '佐吉', '二つ', '年下', '青年', '酒屋', '開く', '居る', '佐吉', '兄さん', '沼津', '大きい', '造酒', '屋', '営む', '佐吉', '末っ子', '知合い']


JSONファイル `train.json` には、データセット $D$ が納められています。
ただし、各小説の本文は既に重要語リストに変換されています。
以下のコードを実行して、データセット $D$ を辞書 `Dw` に読み込んでください。

`Dw` のキーが著者 `author` のとき、`Dw[author]` は、
著者 `author` のすべての小説の重要語リストを連結して得られるリストを与えます。
ここで、著者とは、冒頭で述べた6人の著者のいずれかです。

In [ ]:
### Execute the following code to load the aozora dataset:

Dw = {}
with open('train.json', 'r', encoding='utf-8') as f:
    Dw = json.load(f)

著者ごとに最初の100個の重要語を表示してみましょう。

In [ ]:
for author, words in Dw.items():
    print('著者：', author)
    print('重要語リスト:', words[:100])

著者： 太宰治
重要語リスト: ['創作', '余談', '編輯', '手紙', 'しるす', '在る', 'てれくさい', '語調', 'てれくさい', '作者', '作者', '無名', '創作', '余談', '創作', '自体', '見失う', '追いかける', '思案', '背中', 'むける', '起き直る', '読書', '憤激', '巷', '彷徨', '詩', '篇', 'お話', '甘ったれる', '文学', '書生', '状態', 'ゆえ', '創作', '余談', 'れい', '苦心', '談', '綴る', '器用', '真似', 'できる', 'できる', 'できる', '無理', '文壇', '常識', '破る', '頑固', '信じる', '常識', '従う', '常識', 'ごと', '飛躍', '世', '諸', '現象', '把握', 'ヘエゲル', '支持', 'ほんとう', 'マルクス', 'エンゲルス', '両', 'レニン', '作者', '元来', '言行', '一致', '奇妙', 'こだわる', '作者', '元来', '非', '惨い', '愛する', '趣味', '安心立命', '境地', 'すべて', '崩壊', '前提', 'なす', 'あと', '言葉', '諸兄', 'つづける', '給う', '作者', 'ものぐさ', 'ずるい', '煮る', '焼く', '食える', '境地', '達す', '憎い', '憎い']
著者： 宮沢賢治
重要語リスト: ['虔', '繩', '帯', 'しめる', 'わらう', '杜', '畑', '間', '雨', '青い', '藪', 'よろこぶ', '青', '翔ける', '鷹', '見付ける', 'はねあがる', '知らせる', '虔', 'ばか', '笑', 'ふむ', '虔', '笑', 'ふり', '吹く', 'ぶる', '葉', '光る', 'とき', '虔', 'うれしい', 'うれしい', '笑', '仕方', '無理やり', '大きい', '口', 'あきる', '息', 'ごまかす', 'いつ', 'ぶる', '木', '見上げる', '大きい', '口', '横', 'わき', '痒い', 'ふり', '指', 'こする', '息', '笑',

### 準備2：`compute_word_frequency(dw)` のコピー

辞書 `Aw` (`Dw`) は、タイトル（カテゴリ）をキー、
重要語リストを値とします。

Ex3-1で定義した関数 `compute_word_frequency(dw)` は、
このような辞書 `dw`（`Aw` や `Dw` など）を受け取って、
重要語 $w$ をキー、`dw` が持つリスト全体における $w$ の頻度を値とする辞書を返します。

以下のセルに `compute_word_frequency(dw)` の定義をコピーしてください。

In [ ]:
def compute_word_frequency(dw):
  f = {}
  for key in dw:
    for value in dw[key]:
      if value in f:
        f[value] += 1
      else:
        f[value] = 1
  return f

### 課題

辞書 `word_frequency` は、単語（文字列）をキー、ある文書におけるその単語の出現回数（非負整数）を値とします。
この値は、統計学における頻度（frequency）に相当します。
辞書 `word_frequency` と、カバー率 `coverage`（0以上1以下の実数）を受け取り、以下のようにして、
頻出単語のリストを返す関数 `extract_frequent_words(word_frequency, coverage)` を定義してください。

* 関数 `extract_frequent_words` は、`word_frequency` にある単語を頻度の降順に選んでいきます。
* 全単語の総頻度に対する選ばれた単語の総頻度の比率が `coverage` 以上になったところで、選ぶことを止めます。
* それまでに選んだ単語を選んだ順にリストにして返します。

なお、辞書 `dic` の値による並び替えは、たとえば以下のようにして行うことができます。
```
### 辞書dicのキーkと値vをvの降順に出力
for k, v in sorted(dic.items(), key=lambda x: x[1], reverse=True):
    print(k,v)
```

参考
- [3-1. 辞書](https://utokyo-ipp.github.io/3/3-1.html)
- [6-2. 高階関数](https://utokyo-ipp.github.io/6/6-2.html?highlight=sorted)

In [7]:
##########################################################
##  <[ project1-2-extract_frequent_words ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

QUESTION_EXISTS = False # 質問がある場合は True に変えて，このセル内のコメントとして質問を記述してください
                        # Change to True if you have questions, and describe them as comments in this cell

def extract_frequent_words(word_frequency, coverage):
  total_f = 0
  for k,v in word_frequency.items():
    total_f += v

  c_w_f = 0
  result = []

  for k, v in sorted(word_frequency.items(), key=lambda x: x[1], reverse=True):
    if c_w_f/total_f >= coverage:
      break
    result.append(k)
    c_w_f += v
  return result

提出前に以下のテストセルを実行し、エラーがでないことを確認してください。

In [8]:
word_frequency_for_test = {'私':2, 'あなた':4, '愛':3, '別れ':1}
assert extract_frequent_words(word_frequency_for_test, 0.0) == []
assert extract_frequent_words(word_frequency_for_test, 0.39) == ['あなた']
assert extract_frequent_words(word_frequency_for_test, 0.69) == ['あなた','愛']
assert extract_frequent_words(word_frequency_for_test, 0.89) == ['あなた','愛','私']
assert extract_frequent_words(word_frequency_for_test, 0.99) == ['あなた','愛','私','別れ']
assert extract_frequent_words({}, 1) == []
assert extract_frequent_words({'a':1}, 0.0) == []

## Project1-3. 文書のベクトル化

単語（文字列）のリストとして表現された文書 `words` から、
頻出単語（文字列）のリスト `frequent_words` にしたがって頻度を格納したリスト `vec` を構成することを考えます。
`vec` は、`words` 中における単語 `frequent_words[i]` の頻度を要素 `vec[i]` として持つリストで、`vec` は文書ベクトルと呼ばれます。
（`frequent_words` は他の文書などから既に求められていると考えてください。）

たとえば、`words` が `['今日','雨','明日','雨','明後日','曇り']`、
`frequent_words` が `['晴れ','曇り','雨']` のとき、
`vec` は `[0, 1, 2]` になります。

`words` と `frequent_words` を受け取って、
`vec` を返す関数 `words2vec(words, frequent_words)` を以下のアルゴリズムにしたがって定義してください。

* 空の辞書 `counter` を作り、`frequent_words` の各単語 `fw` について `counter[fw]=0` として初期化します。
* `words` の中の単語 `w` を順に調べて、`w` が `counter` に登録されていれば `counter[w]` を `1` 増やすことにより、`counter[w]` が `words` における `w` の頻度に等しくなるようにします。
* 最後に、`vec[i]` が `counter[frequent_words[i]]` に等しくなるように `vec` を作って返します。

このアルゴリズムは、`words` の長さと `frequent_words` の長さの**和**に、おおよそ比例する時間がかかります。

一方、別のアルゴリズム、たとえば `words.count` を繰り返すものだと、`words` の長さと `frequent_words` の長さの**積**に、おおよそ比例する時間がかかります。
なぜなら、一回の `words.count` の処理には `words` の長さに比例する時間がかかるからです。
同様に、リストに対する `in` 演算子はリストの長さに比例した時間がかかるので、その使用は避けるべきです。

計算結果だけでなく計算時間についても、上で与えたアルゴリズムと同等に定義してください。

In [ ]:
##########################################################
##  <[ project1-3-word2vec ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

QUESTION_EXISTS = False # 質問がある場合は True に変えて，このセル内のコメントとして質問を記述してください
                        # Change to True if you have questions, and describe them as comments in this cell

def words2vec(words, frequent_words):
  counter = {x: 0 for x in frequent_words}
  for w in words:
    if w in counter:
      counter[w] += 1
  vec = [v for v in counter.values()]
  return vec

提出前に以下のテストセルを実行し、エラーがでないことを確認してください。

In [ ]:
assert words2vec(['a','b','c','d','f','a','a','a','b','d','d'], ['a','b','c','d','e']) == [4, 2, 1, 3, 0]
assert words2vec(['今日','雨','明日','雨','明後日','曇り'], ['晴れ','曇り','雨']) == [0, 1, 2]

# 計算結果と計算時間を返す
def time_test(n, m):
    import time
    ws = ['spam'] * n
    fws = ['eggs' + str(i) for i in range(m)]
    s = time.time()
    vec = words2vec(ws, fws)
    e = time.time()
    return (vec, e-s)

# time_test(n, m) の計算時間は n + m におおよそ比例する
for i in range(5):
    c = 4 # 比例定数
    n = 10000 * 2**i
    r1, t1 = time_test(n, 1)
    r2, t2 = time_test(1, n)
    r3, t3 = time_test(n, n)
    if t3 < c*(t1 + t2):
        print(True)
    else:
        # time_test(n, n) が期待される時間内に終わらなかった
        print('Timeout:', t3, '[s]')

True
True
True
True
True


## Project1-4. 著者の推測

### 準備：文書ベクトルの作成

Ex3-1 の `compute_word_frequency` とProject1-2の `extract_frequent_words` とProject1-3の `words2vec` を用いて、`Aw` と `Dw` から `Av` と `Dv` を構成します。

In [ ]:
### Given code:

W = compute_word_frequency(Dw)
F = extract_frequent_words(W, 0.5)

Av = {}
for title, words in Aw.items():
    Av[title] = words2vec(words, F)

Dv = {}
for cate, words in Dw.items():
    Dv[cate] = words2vec(words, F)

### 課題

著者が未知の小説について、
6名の著者のうち、文書ベクトルが最も類似する著者をその（著者未知の）小説の著者と推測します。
ここで、類似度は文書ベクトル間のコサイン類似度により評価します。   

著者をキー、文書ベクトルを値とする辞書 `dv` と、
文書ベクトル `v` をもらって、
文書ベクトルと `v` のコサイン類似度が最も大きい著者を返す関数 `guess_category(dv,v)` を定義してください。
ここで、`dv` は、`Dv` と同じとは限らない、任意の小説に対する文書ベクトルを持つと仮定してください。
また、類似度が最も大きい著者が複数あるときは、どれを返してもよいです。

ベクトル`x`と`y`のコサイン類似度を求める関数 `cos_sim(x, y)` は、第5回の予習課題 (pre5) で定義しました。
以下のセルにその定義をコピーしてください。

数のリスト `ns` を `cos_sim` に渡す際には、十分な計算精度を得るために
`na = np.array(ns, dtype=np.float64)` などとして、`float64` のNumpy配列 `na` に変換してください。

In [ ]:
### Code excerpted from pre5-2
import numpy as np

def cos_sim(x, y):
  x = np.array(x, dtype=np.float64)
  y = np.array(y, dtype=np.float64)
  cos_sim = np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y))
  return cos_sim

In [ ]:
##########################################################
##  <[ project1-4-guess_category ]> 解答セル (Answer cell)
##  このコメントの書き変えを禁ず (Never edit this comment)
##########################################################

QUESTION_EXISTS = False # 質問がある場合は True に変えて，このセル内のコメントとして質問を記述してください
                        # Change to True if you have questions, and describe them as comments in this cell

def guess_category(dv,v):
  max_sim = 0
  result = ''
  for k, vector in dv.items():
    sim = cos_sim(vector,v)
    if max_sim < sim:
      max_sim = sim
      result = k
  return result

提出前に以下のテストセルを実行し、エラーがでないことを確認してください。

In [ ]:
assert guess_category(Dv,Av['老ハイデルベルヒ']) == '太宰治'
assert guess_category(Dv,Av['八十八夜']) == '太宰治'
assert guess_category(Dv,Av['毒もみのすきな署長さん']) == '宮沢賢治'
assert guess_category(Dv,Av['タネリはたしかにいちにち噛んでいたようだった']) == '宮沢賢治'
assert guess_category(Dv,Av['永日小品']) == '夏目漱石'
assert guess_category(Dv,Av['變な音']) == '芥川龍之介' # 夏目漱石
assert guess_category(Dv,Av['蛙']) == '林芙美子'
assert guess_category(Dv,Av['恋愛の微醺']) == '林芙美子'
assert guess_category(Dv,Av['おぎん']) == '芥川龍之介'
assert guess_category(Dv,Av['闇中問答']) == '芥川龍之介'
assert guess_category(Dv,Av['ぐうたら戦記']) == '坂口安吾'
assert guess_category(Dv,Av['桜枝町その他']) == '芥川龍之介' # 坂口安吾